In [1]:
%matplotlib inline
from matplotlib import pyplot as plt

plt.rcParams['figure.figsize'] = (10, 8)
import collections

import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier, export_graphviz

#Синтезируем датасет

In [2]:
#Создание датафрейма с dummy variables
def create_df(dic, feature_list):
  out = pd.DataFrame(dic)
  out = pd.concat([out, pd.get_dummies(out[feature_list])], axis = 1)
  out.drop(feature_list, axis = 1, inplace = True)
  return out

# Некоторые значения есть в тесте, но нет в трейне и наоборот
def intersect_features(train, test):
  common_feat = list(set(train.keys()) & set(test.keys()))
  return train[common_feat], test[common_feat]

In [3]:
features = ['Внешность', 'Алкоголь_в_напитке', 'Уровень_красноречия', 'Потраченные_деньги']

**Обучающая выборка**

In [4]:
df_train = {}
df_train['Внешность'] = ['приятная', 'приятная', 'приятная', 'отталкивающая', 'отталкивающая', 'отталкивающая', 'приятная'] 
df_train['Алкоголь_в_напитке'] = ['да', 'да', 'нет', 'нет', 'да', 'да', 'да']
df_train['Уровень_красноречия'] = ['высокий', 'низкий', 'средний', 'средний', 'низкий', 'высокий', 'средний']
df_train['Потраченные_деньги'] = ['много', 'мало', 'много', 'мало', 'много', 'много', 'много']
df_train['Поедет'] = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])
y_train = LabelEncoder().fit_transform(['+', '-', '+', '-', '-', '+', '+'])

df_train = create_df(df_train, features)
df_train

,Поедет,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_низкий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,0,1,1,0,1,0,0,0,1
1,1,0,1,1,0,0,1,0,1,0
2,0,0,1,0,1,0,0,1,0,1
3,1,1,0,0,1,0,0,1,1,0
4,1,1,0,1,0,0,1,0,0,1
5,0,1,0,1,0,1,0,0,0,1
6,0,0,1,1,0,0,0,1,0,1


**test dataset**

In [5]:
df_test = {}
df_test['Внешность'] = ['приятная', 'приятная', 'отталкивающая'] 
df_test['Алкоголь_в_напитке'] = ['нет', 'да', 'да']
df_test['Уровень_красноречия'] = ['средний', 'высокий', 'средний']
df_test['Потраченные_деньги'] = ['много', 'мало', 'много']
df_test = create_df(df_test, features)
df_test

,Внешность_отталкивающая,Внешность_приятная,Алкоголь_в_напитке_да,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Уровень_красноречия_средний,Потраченные_деньги_мало,Потраченные_деньги_много
0,0,1,0,1,0,1,0,1
1,0,1,1,0,1,0,1,0
2,1,0,1,0,0,1,0,1


In [6]:
# Некоторые значения признаков есть в тесте, но нет в трейне и наоборот
y = df_train['Поедет']
df_train, df_test = intersect_features(train=df_train, test=df_test)
df_train

,Внешность_приятная,Уровень_красноречия_средний,Внешность_отталкивающая,Потраченные_деньги_мало,Потраченные_деньги_много,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Алкоголь_в_напитке_да
0,1,0,0,0,1,0,1,1
1,1,0,0,1,0,0,0,1
2,1,1,0,0,1,1,0,0
3,0,1,1,1,0,1,0,0
4,0,0,1,0,1,0,0,1
5,0,0,1,0,1,0,1,1
6,1,1,0,0,1,0,0,1


In [7]:
df_test

,Внешность_приятная,Уровень_красноречия_средний,Внешность_отталкивающая,Потраченные_деньги_мало,Потраченные_деньги_много,Алкоголь_в_напитке_нет,Уровень_красноречия_высокий,Алкоголь_в_напитке_да
0,1,1,0,0,1,1,0,0
1,1,0,0,1,0,0,1,1
2,0,1,1,0,1,0,0,1


Какова энтропия начальной системы (S0)? Под состояниями системы понимаем значения признака "Поедет" – 0 или 1 (то есть всего 2 состояния).

In [8]:
import math

In [34]:
S = - 4./7. * math.log(2, 4./7.) - 3./7. * math.log(2, 3./7.)
S 

1.058379171527203

Рассмотрим разбиение обучающей выборки по признаку "Внешность_приятная". Какова энтропия S1 левой группы, тех, у кого внешность приятная, и правой группы – S2? Каков прирост информации при данном разбиении (IG)?

In [35]:
S1 = -((3./4.) * math.log(2, 3./4.) + (1./4.) * math.log(2, 1./4.))
S1

1.9320656297399068

In [36]:
S2 = -((1./3.) * math.log(2, 1./3.) + (2./3.) * math.log(2, 2./3.))
S2

1.3499841120914553

In [37]:
IG = S - 4./7. * S1 - 3./7. * S2
IG

-0.6242229506490815

#Постройте с помощью sklearn дерево решений, обучив его на обучающей выборке. Глубину можно не ограничивать.

In [13]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype
---  ------                       --------------  -----
 0   Внешность_приятная           7 non-null      uint8
 1   Уровень_красноречия_средний  7 non-null      uint8
 2   Внешность_отталкивающая      7 non-null      uint8
 3   Потраченные_деньги_мало      7 non-null      uint8
 4   Потраченные_деньги_много     7 non-null      uint8
 5   Алкоголь_в_напитке_нет       7 non-null      uint8
 6   Уровень_красноречия_высокий  7 non-null      uint8
 7   Алкоголь_в_напитке_да        7 non-null      uint8
dtypes: uint8(8)
memory usage: 184.0 bytes


In [14]:
df_train['Внешность_приятная']

0    1
1    1
2    1
3    0
4    0
5    0
6    1
Name: Внешность_приятная, dtype: uint8

In [15]:
y_train = y_train.reshape(-1, 1)

In [16]:
df_train['Внешность_приятная'].values.reshape(-1, 1)

array([[1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [1]], dtype=uint8)

In [18]:
tree = DecisionTreeClassifier(random_state=17)
tree.fit(df_train['Внешность_приятная'].values.reshape(-1, 1), y_train)

export_graphviz(tree, feature_names=['Внешность_приятная'], 
out_file='age_tree.dot', filled=True)
!dot -Tpng 'age_tree.dot' -o 'age_tree.png'

посмотрим на точность

In [21]:
df_test['Внешность_приятная'].values.reshape(-1, 1)

array([[1],
       [1],
       [0]], dtype=uint8)

In [20]:
tree_pred = tree.predict(df_test['Внешность_приятная'].values.reshape(-1, 1))
tree_pred

array([0, 0, 1])

#Функции для расчета энтропии и прироста информации

1 - blue; 0 - yellow

In [22]:
balls = [1 for i in range(9)] + [0 for i in range(11)]

In [23]:
balls_left = [1 for i in range(8)] + [0 for i in range(5)]
balls_right = [1 for i in range(1)] + [0 for i in range(6)]

In [32]:
def entropy(a_list):
  list = np.unique(a_list)
  if len(list) != 1:
    S = 0
    for i in list:
      num = float(a_list.count(i))
      S -= (num/float(len(a_list)) * math.log2(num/float(len(a_list))))

    return S
  else:
    return 0


In [33]:
print(entropy(balls))
print(entropy(balls_left))
print(entropy(balls_right))
print(entropy([1, 2, 3, 4, 5, 6]))

0.9927744539878083
0.9612366047228759
0.5916727785823275
2.584962500721156


In [40]:
# расчет прироста информации

def information_gain(root, left, right):
    ''' root - изначальный набор данных, left и right два разбиения изначального набора'''
    S0 = entropy(root)
    S1 = entropy(left)
    S2 = entropy(right)

    IG = S0 - float(len(left))/float(len(root)) * S1 - float(len(right))/float(len(root)) * S2

    return IG

In [41]:
print(information_gain(balls, balls_left, balls_right))

0.16088518841412436


In [ ]:
def best_feature_to_split(X, y):
    ''' Выводит прирост информации при разбиении по каждому признаку'''
    
    # Ваш код здесь
    pass

#Часть 3. Набор данных "Adult"